In [5]:
import pandas as pd

from datasets import load_dataset

/Users/richardgurtsiev/Desktop/projects/save/oblivion/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
lakera = 'Lakera/gandalf-rct-did'



In [19]:
dataset = load_dataset(lakera)


df_lakera = pd.DataFrame(dataset['defense_in_depth_experiment'])

df_lakera.shape

(107088, 8)